In [ ]:
# Pandas

import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['mpg', 'cyl', 'disp', 'hp', 'weight', 'acc', 'year', 'origin'] 

df = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
df

In [ ]:
df['mpg']

In [ ]:
df['mpg'].values[:5]

In [ ]:
df[['mpg', 'hp']]

In [ ]:
df[['mpg', 'hp']].values[:5]

In [ ]:
df.iloc[:5]

In [ ]:
cond = (df['year'] == 70)
df[cond]

In [ ]:
# Train-Validation-Test Split: Shuffling

df['year'].values[:50]

In [ ]:
shuffled = df.sample(frac=1, random_state=1).reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

trainval, test = train_test_split(shuffled, test_size=0.16, shuffle=False)
train, val = train_test_split(trainval, test_size=0.2, shuffle=False)

In [ ]:
# Cleaning Data

is_missing_attr = train.isna()
n_missing_attr = is_missing_attr.sum(axis=1)
train[n_missing_attr > 0]

In [ ]:
train.dropna(inplace=True)
train

In [ ]:
val.dropna(inplace=True)
test.dropna(inplace=True)